In [1]:
# Olist E-Commerce + Marketing Funnel Dataset - Initial Exploration
# Phase 1: Data Discovery and Business Understanding

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import zipfile
import shutil
import json
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8')

print("=== OLIST COMBINED DATASET EXPLORATION ===")
print("Goal: Understand data structure, relationships, and business opportunities")
print("Datasets: Brazilian E-Commerce + Marketing Funnel")
print("\n" + "="*60 + "\n")


=== OLIST COMBINED DATASET EXPLORATION ===
Goal: Understand data structure, relationships, and business opportunities
Datasets: Brazilian E-Commerce + Marketing Funnel




In [2]:
# =============================================================================
# DATA SETUP: KAGGLE API DOWNLOAD AND FOLDER CREATION
# =============================================================================

def setup_kaggle_datasets():
    """
    Download and setup Olist datasets from Kaggle using API
    This function will:
    1. Check if kaggle.json exists and set up authentication
    2. Create necessary folder structure
    3. Download datasets if not already present
    4. Extract and organize files
    """
    
    print("=== KAGGLE DATASET SETUP ===")
    print("Setting up Olist E-commerce and Marketing Funnel datasets...")
    print("-" * 60)
    
    # Check if kaggle.json exists
    kaggle_json_path = Path('../kaggle.json')
    if not kaggle_json_path.exists():
        print("❌ Error: kaggle.json file not found!")
        print("Please ensure your Kaggle API credentials are in kaggle.json")
        return False
    
    # Set up Kaggle authentication
    kaggle_dir = Path.home() / '.kaggle'
    kaggle_dir.mkdir(exist_ok=True)
    
    # Copy kaggle.json to ~/.kaggle/ if not already there
    target_kaggle_json = kaggle_dir / 'kaggle.json'
    if not target_kaggle_json.exists():
        shutil.copy2(kaggle_json_path, target_kaggle_json)
        # Set proper permissions (required by Kaggle API)
        os.chmod(target_kaggle_json, 0o600)
        print("✓ Kaggle credentials configured")
    
    # Create data directory structure
    data_dir = Path('./data')
    ecommerce_dir = data_dir / 'brazilian-ecommerce'
    marketing_dir = data_dir / 'marketing-funnel'
    
    # Create directories
    ecommerce_dir.mkdir(parents=True, exist_ok=True)
    marketing_dir.mkdir(parents=True, exist_ok=True)
    print(f"✓ Created directory structure: {data_dir}")
    
    # Import kaggle after setting up credentials
    try:
        import kaggle
        from kaggle.api.kaggle_api_extended import KaggleApi
        api = KaggleApi()
        api.authenticate()
        print("✓ Kaggle API authenticated successfully")
    except ImportError:
        print("❌ Error: kaggle package not installed!")
        print("Install with: pip install kaggle")
        return False
    except Exception as e:
        print(f"❌ Error authenticating with Kaggle API: {e}")
        return False
    
    # Dataset configurations
    datasets = {
        'brazilian-ecommerce': {
            'dataset': 'olistbr/brazilian-ecommerce',
            'target_dir': ecommerce_dir,
            'expected_files': [
                'olist_customers_dataset.csv',
                'olist_orders_dataset.csv',
                'olist_order_items_dataset.csv',
                'olist_order_payments_dataset.csv',
                'olist_order_reviews_dataset.csv',
                'olist_products_dataset.csv',
                'olist_sellers_dataset.csv',
                'olist_geolocation_dataset.csv',
                'product_category_name_translation.csv'
            ]
        },
        'marketing-funnel': {
            'dataset': 'olistbr/marketing-funnel-olist',
            'target_dir': marketing_dir,
            'expected_files': [
                'olist_marketing_qualified_leads_dataset.csv',
                'olist_closed_deals_dataset.csv'
            ]
        }
    }
    
    # Download and extract datasets
    for dataset_name, config in datasets.items():
        print(f"\n--- Processing {dataset_name} ---")
        
        # Check if files already exist
        existing_files = [f for f in config['expected_files'] 
                         if (config['target_dir'] / f).exists()]
        
        if len(existing_files) == len(config['expected_files']):
            print(f"✓ All files already exist for {dataset_name}")
            # Show file sizes for verification
            for file in config['expected_files']:
                file_path = config['target_dir'] / file
                size_mb = file_path.stat().st_size / (1024 * 1024)
                print(f"  - {file}: {size_mb:.1f} MB")
            continue
        
        print(f"📥 Downloading {dataset_name} dataset...")
        try:
            # Download dataset
            api.dataset_download_files(
                config['dataset'], 
                path=config['target_dir'], 
                unzip=True,
                quiet=False
            )
            print(f"✓ Downloaded and extracted {dataset_name}")
            
            # Verify files
            missing_files = [f for f in config['expected_files'] 
                           if not (config['target_dir'] / f).exists()]
            
            if missing_files:
                print(f"⚠️  Warning: Some expected files are missing from {dataset_name}:")
                for file in missing_files:
                    print(f"  - {file}")
            else:
                print(f"✓ All expected files present for {dataset_name}")
                # Show file sizes
                for file in config['expected_files']:
                    file_path = config['target_dir'] / file
                    if file_path.exists():
                        size_mb = file_path.stat().st_size / (1024 * 1024)
                        print(f"  - {file}: {size_mb:.1f} MB")
            
        except Exception as e:
            print(f"❌ Error downloading {dataset_name}: {e}")
            return False
    
    # Clean up any zip files that might be left
    for zip_file in data_dir.rglob("*.zip"):
        try:
            zip_file.unlink()
            print(f"🗑️  Cleaned up {zip_file.name}")
        except:
            pass
    
    print("\n" + "="*60)
    print("✅ DATASET SETUP COMPLETE!")
    print("All datasets are ready for analysis.")
    print("="*60)
    
    return True

# Run the setup
setup_success = setup_kaggle_datasets()

if not setup_success:
    print("\n❌ Setup failed. Please check the error messages above.")
    print("Common solutions:")
    print("1. Ensure kaggle.json exists in the current directory")
    print("2. Install kaggle package: pip install kaggle")
    print("3. Check your internet connection")
    print("4. Verify your Kaggle API credentials are valid")

=== KAGGLE DATASET SETUP ===
Setting up Olist E-commerce and Marketing Funnel datasets...
------------------------------------------------------------
✓ Kaggle credentials configured
✓ Created directory structure: data
✓ Kaggle API authenticated successfully

--- Processing brazilian-ecommerce ---
📥 Downloading brazilian-ecommerce dataset...
Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce


100%|██████████| 42.6M/42.6M [00:00<00:00, 2.64GB/s]

✓ Downloaded and extracted brazilian-ecommerce
✓ All expected files present for brazilian-ecommerce
  - olist_customers_dataset.csv: 8.6 MB
  - olist_orders_dataset.csv: 16.8 MB
  - olist_order_items_dataset.csv: 14.7 MB
  - olist_order_payments_dataset.csv: 5.5 MB
  - olist_order_reviews_dataset.csv: 13.8 MB
  - olist_products_dataset.csv: 2.3 MB
  - olist_sellers_dataset.csv: 0.2 MB
  - olist_geolocation_dataset.csv: 58.4 MB
  - product_category_name_translation.csv: 0.0 MB

--- Processing marketing-funnel ---
📥 Downloading marketing-funnel dataset...
Dataset URL: https://www.kaggle.com/datasets/olistbr/marketing-funnel-olist


100%|██████████| 278k/278k [00:00<00:00, 760MB/s]


✓ Downloaded and extracted marketing-funnel
✓ All expected files present for marketing-funnel
  - olist_marketing_qualified_leads_dataset.csv: 0.7 MB
  - olist_closed_deals_dataset.csv: 0.2 MB

✅ DATASET SETUP COMPLETE!
All datasets are ready for analysis.


In [3]:
# =============================================================================
# STEP 1: DATA INVENTORY AND LOADING
# =============================================================================

print("STEP 1: DATA INVENTORY AND LOADING")
print("-" * 40)

# Define file paths - adjust these to your local paths
ecommerce_path = Path('./data/brazilian-ecommerce/')
marketing_path = Path('./data/marketing-funnel/')

# E-Commerce dataset files
ecommerce_files = {
    'customers': 'olist_customers_dataset.csv',
    'orders': 'olist_orders_dataset.csv', 
    'order_items': 'olist_order_items_dataset.csv',
    'order_payments': 'olist_order_payments_dataset.csv',
    'order_reviews': 'olist_order_reviews_dataset.csv',
    'products': 'olist_products_dataset.csv',
    'sellers': 'olist_sellers_dataset.csv',
    'geolocation': 'olist_geolocation_dataset.csv',
    'category_translation': 'product_category_name_translation.csv'
}

# Marketing Funnel dataset files
marketing_files = {
    'marketing_qualified_leads': 'olist_marketing_qualified_leads_dataset.csv',
    'closed_deals': 'olist_closed_deals_dataset.csv'
}

# Load all datasets
print("Loading E-Commerce datasets...")
ecommerce_data = {}
for name, filename in ecommerce_files.items():
    try:
        df = pd.read_csv(ecommerce_path / filename)
        ecommerce_data[name] = df
        print(f"✓ {name}: {df.shape[0]:,} rows x {df.shape[1]} cols")
    except FileNotFoundError:
        print(f"✗ {name}: File not found - {filename}")

print("\nLoading Marketing Funnel datasets...")
marketing_data = {}
for name, filename in marketing_files.items():
    try:
        df = pd.read_csv(marketing_path / filename)
        marketing_data[name] = df
        print(f"✓ {name}: {df.shape[0]:,} rows x {df.shape[1]} cols")
    except FileNotFoundError:
        print(f"✗ {name}: File not found - {filename}")

STEP 1: DATA INVENTORY AND LOADING
----------------------------------------
Loading E-Commerce datasets...
✓ customers: 99,441 rows x 5 cols
✓ orders: 99,441 rows x 8 cols
✓ order_items: 112,650 rows x 7 cols
✓ order_payments: 103,886 rows x 5 cols
✓ order_reviews: 99,224 rows x 7 cols
✓ products: 32,951 rows x 9 cols
✓ sellers: 3,095 rows x 4 cols
✓ geolocation: 1,000,163 rows x 5 cols
✓ category_translation: 71 rows x 2 cols

Loading Marketing Funnel datasets...
✓ marketing_qualified_leads: 8,000 rows x 4 cols
✓ closed_deals: 842 rows x 14 cols


In [12]:
# =============================================================================
# STEP 2: INITIAL DATA PROFILING
# =============================================================================

print("\n" + "="*60)
print("STEP 2: INITIAL DATA PROFILING")

def profile_dataset(df, name):
    """Quick data profiling function"""
    print(f"\n=== {name.upper()} PROFILE ===")
    print(f"Shape: {df.shape}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Display columns as bulleted list
    print(f"Columns ({len(df.columns)} total):")
    for col in df.columns:
        print(f"  • {col}")
    
    # Null values
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        print(f"Null values:")
        for col, count in null_counts[null_counts > 0].items():
            print(f"  {col}: {count:,} ({count/len(df)*100:.1f}%)")
    else:
        print("No null values found")
    
    # Data types
    print(f"Data types: {df.dtypes.value_counts().to_dict()}")
    
    return df.describe(include='all')

# Profile ALL datasets from both e-commerce and marketing collections
print("\n=== PROFILING ALL E-COMMERCE DATASETS ===")
ecommerce_profiles = {}
for dataset_name, dataset_df in ecommerce_data.items():
    print(f"\n{'='*60}")
    ecommerce_profiles[dataset_name] = profile_dataset(dataset_df, f"E-Commerce: {dataset_name}")

print("\n\n=== PROFILING ALL MARKETING DATASETS ===")
marketing_profiles = {}
for dataset_name, dataset_df in marketing_data.items():
    print(f"\n{'='*60}")
    marketing_profiles[dataset_name] = profile_dataset(dataset_df, f"Marketing: {dataset_name}")

print(f"\n{'='*60}")
print(f"SUMMARY: Profiled {len(ecommerce_profiles)} e-commerce datasets and {len(marketing_profiles)} marketing datasets")
print(f"E-commerce datasets: {list(ecommerce_profiles.keys())}")
print(f"Marketing datasets: {list(marketing_profiles.keys())}")


STEP 2: INITIAL DATA PROFILING

=== PROFILING ALL E-COMMERCE DATASETS ===


=== E-COMMERCE: CUSTOMERS PROFILE ===
Shape: (99441, 5)
Memory usage: 29.62 MB
Columns (5 total):
  • customer_id
  • customer_unique_id
  • customer_zip_code_prefix
  • customer_city
  • customer_state
No null values found
Data types: {dtype('O'): 4, dtype('int64'): 1}


=== E-COMMERCE: ORDERS PROFILE ===
Shape: (99441, 8)
Memory usage: 58.97 MB
Columns (8 total):
  • order_id
  • customer_id
  • order_status
  • order_purchase_timestamp
  • order_approved_at
  • order_delivered_carrier_date
  • order_delivered_customer_date
  • order_estimated_delivery_date
Null values:
  order_approved_at: 160 (0.2%)
  order_delivered_carrier_date: 1,783 (1.8%)
  order_delivered_customer_date: 2,965 (3.0%)
Data types: {dtype('O'): 8}


=== E-COMMERCE: ORDER_ITEMS PROFILE ===
Shape: (112650, 7)
Memory usage: 39.43 MB
Columns (7 total):
  • order_id
  • order_item_id
  • product_id
  • seller_id
  • shipping_limit_date
  • pr

In [8]:
# =============================================================================
# STEP 3: UNDERSTANDING THE MARKETING FUNNEL CONNECTION
# =============================================================================

print("\n" + "="*60)
print("STEP 3: MARKETING FUNNEL CONNECTION ANALYSIS")
print("-" * 40)

# Based on the Kaggle joining instructions, the connection is:
# Marketing Qualified Leads -> Closed Deals -> Sellers -> Orders

if 'marketing_qualified_leads' in marketing_data and 'closed_deals' in marketing_data:
    mql_df = marketing_data['marketing_qualified_leads']
    deals_df = marketing_data['closed_deals']
    
    print("Marketing Funnel Structure:")
    print(f"1. Marketing Qualified Leads: {mql_df.shape[0]:,} leads")
    print(f"2. Closed Deals: {deals_df.shape[0]:,} deals")
    
    # Check the connection between MQL and Deals
    if 'mql_id' in mql_df.columns and 'mql_id' in deals_df.columns:
        mql_to_deals = deals_df['mql_id'].nunique()
        conversion_rate = mql_to_deals / mql_df.shape[0] * 100
        print(f"3. MQL to Deal Conversion: {mql_to_deals:,} converted ({conversion_rate:.1f}%)")
    
    # Check seller connection
    if 'seller_id' in deals_df.columns and 'sellers' in ecommerce_data:
        sellers_df = ecommerce_data['sellers']
        marketing_sellers = deals_df['seller_id'].nunique()
        total_sellers = sellers_df['seller_id'].nunique()
        print(f"4. Sellers from Marketing: {marketing_sellers:,} out of {total_sellers:,} total sellers")


STEP 3: MARKETING FUNNEL CONNECTION ANALYSIS
----------------------------------------
Marketing Funnel Structure:
1. Marketing Qualified Leads: 8,000 leads
2. Closed Deals: 842 deals
3. MQL to Deal Conversion: 842 converted (10.5%)
4. Sellers from Marketing: 842 out of 3,095 total sellers


In [ ]:
# =============================================================================
# STEP 4: JOIN THE DATASETS
# =============================================================================

print("\n" + "="*60)
print("STEP 4: JOINING E-COMMERCE AND MARKETING DATA")
print("-" * 40)

# Following the Kaggle joining instructions:
# MQL -> Closed Deals -> Sellers -> Order Items -> Orders

if all(key in marketing_data for key in ['marketing_qualified_leads', 'closed_deals']) and \
   all(key in ecommerce_data for key in ['sellers', 'order_items', 'orders']):
    
    print("Creating comprehensive joined dataset...")
    
    # Step 1: Join MQL with Closed Deals
    mql_deals = pd.merge(
        marketing_data['marketing_qualified_leads'],
        marketing_data['closed_deals'],
        on='mql_id',
        how='inner'
    )
    print(f"MQL + Deals: {mql_deals.shape[0]:,} records")
    
    # Step 2: Join with Sellers
    deals_sellers = pd.merge(
        mql_deals,
        ecommerce_data['sellers'],
        on='seller_id',
        how='inner'
    )
    print(f"+ Sellers: {deals_sellers.shape[0]:,} records")
    
    # Step 3: Join with Order Items
    deals_order_items = pd.merge(
        deals_sellers,
        ecommerce_data['order_items'],
        on='seller_id',
        how='inner'
    )
    print(f"+ Order Items: {deals_order_items.shape[0]:,} records")
    
    # Step 4: Join with Orders
    combined_dataset = pd.merge(
        deals_order_items,
        ecommerce_data['orders'],
        on='order_id',
        how='inner'
    )
    print(f"+ Orders: {combined_dataset.shape[0]:,} records")
    
    # Step 5: Add customer information
    if 'customers' in ecommerce_data:
        combined_dataset = pd.merge(
            combined_dataset,
            ecommerce_data['customers'],
            on='customer_id',
            how='left'
        )
        print(f"+ Customers: {combined_dataset.shape[0]:,} records")
    
    # Step 6: Add product information
    if 'products' in ecommerce_data:
        combined_dataset = pd.merge(
            combined_dataset,
            ecommerce_data['products'],
            on='product_id',
            how='left'
        )
        print(f"+ Products: {combined_dataset.shape[0]:,} records")
    
    print(f"\nFinal combined dataset shape: {combined_dataset.shape}")
    print(f"Columns: {combined_dataset.shape[1]}")

In [ ]:
# =============================================================================
# STEP 5: BUSINESS QUESTIONS DISCOVERY
# =============================================================================

print("\n" + "="*60)
print("STEP 5: BUSINESS QUESTIONS DISCOVERY")
print("-" * 40)

if 'combined_dataset' in locals():
    print("Analyzing business opportunities with combined dataset...\n")
    
    # Marketing Attribution Analysis
    print("1. MARKETING ATTRIBUTION OPPORTUNITIES:")
    
    # Lead source effectiveness
    if 'lead_type' in combined_dataset.columns:
        lead_performance = combined_dataset.groupby('lead_type').agg({
            'price': ['count', 'sum', 'mean'],
            'order_id': 'nunique'
        }).round(2)
        print("Lead Type Performance:")
        print(lead_performance.head())
    
    # Marketing channel ROI
    if 'origin' in combined_dataset.columns:
        channel_performance = combined_dataset.groupby('origin').agg({
            'price': ['count', 'sum'],
            'freight_value': 'sum'
        }).round(2)
        print("\nMarketing Channel Performance:")
        print(channel_performance.head())
    
    # 2. Customer Lifecycle Analysis
    print("\n2. CUSTOMER LIFECYCLE OPPORTUNITIES:")
    
    # Time from lead to purchase
    if 'first_contact_date' in combined_dataset.columns and 'order_purchase_timestamp' in combined_dataset.columns:
        combined_dataset['first_contact_date'] = pd.to_datetime(combined_dataset['first_contact_date'])
        combined_dataset['order_purchase_timestamp'] = pd.to_datetime(combined_dataset['order_purchase_timestamp'])
        combined_dataset['lead_to_purchase_days'] = (
            combined_dataset['order_purchase_timestamp'] - combined_dataset['first_contact_date']
        ).dt.days
        
        print(f"Lead to Purchase Time (days):")
        print(combined_dataset['lead_to_purchase_days'].describe())
    
    # 3. Geographic Analysis
    print("\n3. GEOGRAPHIC OPPORTUNITIES:")
    
    if 'seller_state' in combined_dataset.columns and 'customer_state' in combined_dataset.columns:
        # Cross-state sales from marketing
        geo_analysis = combined_dataset.groupby(['seller_state', 'customer_state']).agg({
            'price': 'sum',
            'order_id': 'nunique'
        }).reset_index()
        
        print("Top 10 Seller-Customer State Combinations:")
        top_geo = geo_analysis.nlargest(10, 'price')
        print(top_geo)

In [ ]:
# =============================================================================
# STEP 6: COMPREHENSIVE BUSINESS QUESTIONS LIST
# =============================================================================

print("\n" + "="*60)
print("STEP 6: COMPREHENSIVE BUSINESS QUESTIONS")
print("-" * 40)

business_questions = [
    # Marketing Attribution
    "1. Which marketing channels (lead sources) generate the highest revenue?",
    "2. What is the ROI of different marketing campaigns?",
    "3. How long does it take from first contact to purchase?",
    "4. Which lead types convert to the highest-value customers?",
    
    # Customer Lifecycle
    "5. What is the customer acquisition cost by channel?",
    "6. How does marketing-acquired customer behavior differ from organic customers?",
    "7. What is the lifetime value of marketing-acquired customers?",
    "8. Which marketing touchpoints lead to repeat purchases?",
    
    # Seller Performance
    "9. How do marketing-acquired sellers perform vs. organic sellers?",
    "10. Which seller onboarding channels lead to better long-term performance?",
    "11. What is the seller retention rate by acquisition channel?",
    
    # Product & Category Analysis
    "12. Which product categories benefit most from marketing investment?",
    "13. How does marketing affect seasonal product sales?",
    "14. What products do marketing-acquired customers prefer?",
    
    # Geographic Insights
    "15. How does marketing effectiveness vary by Brazilian state?",
    "16. Which regions have untapped marketing potential?",
    "17. How does shipping cost affect marketing ROI by location?",
    
    # Business Intelligence
    "18. What are the key metrics for marketing campaign optimization?",
    "19. How can we predict which leads will convert to high-value customers?",
    "20. What marketing mix generates the best overall marketplace health?"
]

print("COMPREHENSIVE BUSINESS QUESTIONS WITH COMBINED DATASET:")
for question in business_questions:
    print(question)

In [ ]:
# =============================================================================
# STEP 7: PROPOSED STAR SCHEMA DESIGN
# =============================================================================

print("\n" + "="*60)
print("STEP 7: PROPOSED STAR SCHEMA DESIGN")
print("-" * 40)

print("""
ENHANCED STAR SCHEMA WITH MARKETING FUNNEL DATA:

FACT TABLE: FactSales
- Grain: One row per order item (enhanced with marketing attribution)
- Estimated rows: ~117,000+ (from order_items + marketing attribution)

CORE MEASURES:
- price (unit price)
- freight_value 
- total_value (price + freight)
- lead_to_purchase_days (new!)
- marketing_cost_allocation (new!)

DIMENSION TABLES:

1. DimCustomer
   - customer_id, customer_unique_id
   - customer_city, customer_state, customer_zip_code_prefix
   - acquisition_channel (organic vs marketing)

2. DimProduct  
   - product_id, product_category_name
   - product_name_length, product_description_length
   - product_photos_qty, product_weight_g, product_length_cm, etc.

3. DimSeller (ENHANCED)
   - seller_id, seller_zip_code_prefix, seller_city, seller_state
   - acquisition_channel (organic vs marketing)
   - mql_id, deal_close_date (new marketing fields!)

4. DimDate
   - Date hierarchy for orders and marketing events
   - order_date, estimated_delivery_date, delivered_date
   - first_contact_date, deal_close_date (new!)

5. DimGeography
   - Geographic lookup for Brazilian states/cities
   - Both seller and customer geography

6. DimPayment
   - payment_type, payment_installments
   - payment_value

7. DimMarketing (NEW!)
   - mql_id, lead_type, lead_behaviour_profile
   - origin (marketing channel)
   - first_contact_date, landing_page_id
   - business_segment, business_type

8. DimProduct Category (NEW!)
   - product_category_name (Portuguese)
   - product_category_name_english
   - category_group (custom groupings)

BRIDGE TABLE (Optional):
- BridgeMarketingAttribution
   - Links orders to multiple marketing touchpoints
   - Handles multi-touch attribution

KEY RELATIONSHIPS:
FactSales connects to:
- DimCustomer via customer_key
- DimProduct via product_key  
- DimSeller via seller_key
- DimDate via order_date_key
- DimMarketing via mql_key (for marketing-attributed sales)
- DimPayment via payment_key
""")

In [ ]:
# =============================================================================
# STEP 8: DATA QUALITY ASSESSMENT
# =============================================================================

print("\n" + "="*60)
print("STEP 8: DATA QUALITY ASSESSMENT")
print("-" * 40)

if 'combined_dataset' in locals():
    print("Data Quality Issues Identified:")
    
    # Check for missing values in key fields
    key_fields = ['order_id', 'customer_id', 'seller_id', 'product_id', 'price']
    missing_analysis = {}
    
    for field in key_fields:
        if field in combined_dataset.columns:
            missing_count = combined_dataset[field].isnull().sum()
            missing_pct = missing_count / len(combined_dataset) * 100
            missing_analysis[field] = {'count': missing_count, 'percentage': missing_pct}
            
            if missing_count > 0:
                print(f"⚠️  {field}: {missing_count:,} missing ({missing_pct:.1f}%)")
            else:
                print(f"✓ {field}: No missing values")
    
    # Check for duplicates
    if len(combined_dataset) > 0:
        duplicates = combined_dataset.duplicated().sum()
        print(f"\nDuplicate rows: {duplicates:,}")
        
        # Check business key uniqueness
        if 'order_id' in combined_dataset.columns and 'order_item_id' in combined_dataset.columns:
            business_key_dupes = combined_dataset.duplicated(['order_id', 'order_item_id']).sum()
            print(f"Business key duplicates (order_id + order_item_id): {business_key_dupes:,}")

In [ ]:
# =============================================================================
# STEP 9: IMPLEMENTATION RECOMMENDATIONS
# =============================================================================

print("\n" + "="*60)
print("STEP 9: IMPLEMENTATION RECOMMENDATIONS")
print("-" * 40)

print("""
PHASE 2 IMPLEMENTATION PLAN:

1. DATA PIPELINE SETUP:
   - Use Meltano to ingest both CSV datasets
   - Create staging tables in BigQuery
   - Implement incremental loading for ongoing data

2. DBT TRANSFORMATIONS:
   - Create staging models for each source table
   - Build dimension tables with proper SCD Type 2 for sellers
   - Create fact table with marketing attribution
   - Add data quality tests at each layer

3. GREAT EXPECTATIONS RULES:
   - Order values must be positive
   - All order_items must have valid order_id references
   - Marketing leads must have valid conversion dates
   - Geographic codes must match Brazilian standards
   - Lead-to-purchase time must be reasonable (0-365 days)

4. BUSINESS INTELLIGENCE LAYER:
   - Marketing attribution dashboard
   - Customer acquisition cost analysis
   - Seller performance by acquisition channel
   - Geographic market penetration analysis

5. ADVANCED ANALYTICS:
   - Lead scoring model
   - Customer lifetime value prediction
   - Marketing mix optimization
   - Churn prediction for marketing-acquired customers

EXPECTED DELIVERABLES:
- Star schema with marketing attribution
- Marketing ROI dashboard
- Customer acquisition cost analysis
- Seller onboarding effectiveness report
- Geographic expansion opportunity analysis
""")

print("\n" + "="*60)
print("PHASE 1 EDA COMPLETE!")
print("Next Steps: Proceed to infrastructure setup and implementation")
print("="*60)